<p> Dataset de calidad de vinos </p>
<p>1 - fixed acidity<br>
   2 - volatile acidity<br>
   3 - citric acid<br>
   4 - residual sugar<br>
   5 - chlorides<br>
   6 - free sulfur dioxide<br>
   7 - total sulfur dioxide<br>
   8 - density<br>
   9 - pH<br>
   10 - sulphates<br>
   11 - alcohol<br><br>
   Output variable (based on sensory data):<br> 
   12 - quality (score between 0 and 10)</p>
   
<p>Cita:<br>
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.<br> 
Modeling wine preferences by data mining from physicochemical properties.<br>
In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.</p>


!pip install --upgrade scikit-learn
!pip install yellowbrick

In [ ]:
#Importing required packages.
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Carguemos el dataset
wine = pd.read_csv('winequality-red.csv')

In [ ]:
# Revisemos el dataset
wine.head()

In [ ]:
# Informacion sobre las columnas
wine.info()

### Hagamos analisis de datos exploratorio

Por lo general queremos ver tres cosas:
    1. Distribucion de cada variable
    2. Comparativo de variable objetivo contra variables predictivas
    3. Correlacion entre todas las variables
    
Esto nos permite tomar decisiones en cuando a las transformaciones requeridas para un mejor modelo

#### Grafiquemos la distribucion de las variables

In [ ]:
# Funcion para graficar distribucion
def getBoxplot(data, col):

    # Crear figura y establecer su tamaño
    fig, ax = plt.subplots(figsize=(12,5))

    # Genera un boxplot para cada variable del dataset
    sns.boxplot(data[col], ax=ax)
    
    # Asigna titulo a cada grafico
    plt.title(f"Distribución de {col}")
    
    # Muestra grafico
    plt.show()

    
# Bucle para graficar la distribucion de todas las variables utilizando funcion getBoxplot
for col in wine.columns:
    getBoxplot(wine, col)


#### Grafiquemos todas las variables contra nuestra variable objetivo: quality

In [ ]:
# Funcion para graficar distribucion
def getBarplot(data, col, objetivo = 'quality'):
    
    # Crear figura y establecer su tamaño
    fig, ax = plt.subplots(figsize=(12,5))
    
    # Genera un grafico de barras contra variable objetivo
    sns.barplot(x = objetivo, y = col, data = data, ax=ax)
    
    # Asigna titulo a cada grafico
    plt.title(f"Comparativo {col} vs {objetivo}")
    
    # Muestra grafico
    plt.show()

    
# Bucle para graficar la distribucion de todas las variables utilizando funcion getBoxplot
for col in wine.columns:
    if col != "quality":
        getBarplot(wine, col)


#### Grafiquemos la matriz de correlacion

In [ ]:
# Crear tabla de correlacion
corr = wine.corr()
corr

In [ ]:
# Grafiquemos la matriz de correlacion para tener un 
fig, ax = plt.subplots(figsize=(15,8))

# Preparacion de datos para mostrar triangulo inferior
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Crear heatmap con matriz de correlacion
sns.heatmap(corr, vmin=-1, vmax=1, cmap="RdBu", annot=True, linewidths=.5, fmt=".2f", mask=mask)

# Asignar titulo al grafico
ax.set_title("Mapa de calor: Matriz de correlacion")

plt.show()

### Generemos unos modelos de regresion capaces de predecir la calidad del vino

#### Importemos algunos paquetes

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import scipy

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

#### Dividamos los datos en set de entrenamiento y validacion

In [ ]:
# Asignar variable objetivo a "y" y variables predictivas a "X"
X = wine.drop(columns=['quality', 'pH', 'free sulfur dioxide', 'residual sugar'])
y = wine['quality']

# Dividamos los datos en entrenamiento y validacion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#### Creemos una funcion para evaluar validez del modelo

In [ ]:
from yellowbrick.regressor import ResidualsPlot

# Funcion que grafica diagnostico de regresion
def getDiagnosticsPlots(modelo, X, y):
    
    y_pred = modelo.predict(X)
    resid = y - y_pred

    # Instanciamos figura
    fig, ax = plt.subplots(figsize=(12,10), nrows=2, ncols=1)
    
    # Crear grafico de valores residuales
    ax[0].scatter(y_pred, resid)
    ax[0].set_xlabel("Predicted")
    ax[0].set_ylabel("Residuales")
    ax[0].set_title("Residual plot")

    # Crear grafico QQ Plot
    scipy.stats.probplot(pred_train, dist="norm", plot=ax[1])

    plt.show()


#### Primero intentemos generar algunos modelos sin preprocesar variables

In [ ]:
# Instanciamos regresion linear
reglin = LinearRegression()

# Ajustamos los datos a regresion linear
reglin.fit(X_train, y_train)

# Creamos predicciones sobre set de entrenamiento y validacion
pred_train = reglin.predict(X_train)
pred_test = reglin.predict(X_test)

# Calculamos metricas - r2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f"R2 en datos de entrenamiento: {round(r2_train, 3)}")
print(f"R2 en datos de validacion: {round(r2_test, 3)}")

# Agregamos un espacio para claridad en resultados
print("\n")

# Calculamos metricas - mse
mse_train = mean_squared_error(y_train, pred_train)
mse_test =  mean_squared_error(y_test, pred_test)
print(f"MSE en datos de entrenamiento: {round(mse_train, 3)}")
print(f"MSE en datos de validacion: {round(mse_test, 3)}")
 
# Grafiquemos los resultados de la regresion
print("\nResultados con datos de entrenamiento\n")
getDiagnosticsPlots(reglin, X_train, y_train)

print("\nResultados con datos de validacion\n")
getDiagnosticsPlots(reglin, X_test, y_test)

In [ ]:
# Instanciamos regresion con arbol de decisiones
tree = DecisionTreeRegressor(random_state=42)

# Ajustamos los datos a regresion linear
tree.fit(X_train, y_train)

# Creamos predicciones sobre set de entrenamiento y validacion
pred_train = tree.predict(X_train)
pred_test = tree.predict(X_test)

# Calculamos metricas - r2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f"R2 en datos de entrenamiento: {round(r2_train, 3)}")
print(f"R2 en datos de validacion: {round(r2_test, 3)}")

# Agregamos un espacio para claridad en resultados
print("\n")

# Calculamos metricas - mse
mse_train = mean_squared_error(y_train, pred_train)
mse_test =  mean_squared_error(y_test, pred_test)
print(f"MSE en datos de entrenamiento: {round(mse_train, 3)}")
print(f"MSE en datos de validacion: {round(mse_test, 3)}")

# Grafiquemos los resultados de la regresion
print("\nResultados con datos de entrenamiento\n")
getDiagnosticsPlots(tree, X_train, y_train)

print("\nResultados con datos de validacion\n")
getDiagnosticsPlots(tree, X_test, y_test)

In [ ]:
# Instanciamos regresion con arbol de decisiones
rf = RandomForestRegressor(random_state=42)

# Ajustamos los datos a regresion linear
rf.fit(X_train, y_train)

# Creamos predicciones sobre set de entrenamiento y validacion
pred_train = rf.predict(X_train)
pred_test = rf.predict(X_test)

# Calculamos metricas - r2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f"R2 en datos de entrenamiento: {round(r2_train, 3)}")
print(f"R2 en datos de validacion: {round(r2_test, 3)}")

# Agregamos un espacio para claridad en resultados
print("\n")

# Calculamos metricas - mse
mse_train = mean_squared_error(y_train, pred_train)
mse_test =  mean_squared_error(y_test, pred_test)
print(f"MSE en datos de entrenamiento: {round(mse_train, 3)}")
print(f"MSE en datos de validacion: {round(mse_test, 3)}")

# Grafiquemos los resultados de la regresion
print("\nResultados con datos de entrenamiento\n")
getDiagnosticsPlots(rf, X_train, y_train)

print("\nResultados con datos de validacion\n")
getDiagnosticsPlots(rf, X_test, y_test)

In [ ]:
# Instanciamos regresion con LightGBM
lgb = LGBMRegressor(random_state=42)

# Ajustamos los datos a regresion
lgb.fit(X_train, y_train)

# Creamos predicciones sobre set de entrenamiento y validacion
pred_train = lgb.predict(X_train)
pred_test = lgb.predict(X_test)

# Calculamos metricas - r2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f"R2 en datos de entrenamiento: {round(r2_train, 3)}")
print(f"R2 en datos de validacion: {round(r2_test, 3)}")

# Agregamos un espacio para claridad en resultados
print("\n")

# Calculamos metricas - mse
mse_train = mean_squared_error(y_train, pred_train)
mse_test =  mean_squared_error(y_test, pred_test)
print(f"MSE en datos de entrenamiento: {round(mse_train, 3)}")
print(f"MSE en datos de validacion: {round(mse_test, 3)}")

# Grafiquemos los resultados de la regresion
print("\nResultados con datos de entrenamiento\n")
getDiagnosticsPlots(lgb, X_train, y_train)

print("\nResultados con datos de validacion\n")
getDiagnosticsPlots(lgb, X_test, y_test)

#### Tratemos los outliers

In [ ]:
# Columnas a imputar outliers
to_impute = ["fixed acidity",
"volatile acidity",
"chlorides",
"total sulfur dioxide",
"sulphates"
]


# Iniciar un diccionario para imputar outliers
impute_dict = {col:[] for col in to_impute}
              
# Bucle para obtener quantiles de variables
for col in to_impute:
    impute_dict[col] = np.quantile(X_train[col], .975)

# Bucle para imputar posibles outliers
for col in to_impute:
    
    # Imputar posibles outliers del set de entrenamiento
    ix = X_train[col] > impute_dict[col]
    X_train.loc[ix, col] = impute_dict[col]
    
    # Imputar posibles outliers del set de validacion
    ix = X_test[col] > impute_dict[col]
    X_test.loc[ix, col] = impute_dict[col]


In [ ]:
# Bucle para graficar la distribucion de todas las variables utilizando funcion getBoxplot
for col in X_train.columns:
    getBoxplot(X_train, col)

#### Intentemos modelar con outliers reemplazados

In [ ]:
# Funcion para dejar todas las variables en la misma escala
from sklearn.preprocessing import StandardScaler

In [ ]:
# Dejemos todas las variables en la misma escala
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Instanciamos regresion linear
reglin = LinearRegression()

# Ajustamos los datos a regresion linear
reglin.fit(X_train, y_train)

# Creamos predicciones sobre set de entrenamiento y validacion
pred_train = reglin.predict(X_train)
pred_test = reglin.predict(X_test)

# Calculamos metricas - r2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f"R2 en datos de entrenamiento: {round(r2_train, 3)}")
print(f"R2 en datos de validacion: {round(r2_test, 3)}")

# Agregamos un espacio para claridad en resultados
print("\n")

# Calculamos metricas - mse
mse_train = mean_squared_error(y_train, pred_train)
mse_test =  mean_squared_error(y_test, pred_test)
print(f"MSE en datos de entrenamiento: {round(mse_train, 3)}")
print(f"MSE en datos de validacion: {round(mse_test, 3)}")
 
# Grafiquemos los resultados de la regresion
print("\nResultados con datos de entrenamiento\n")
getDiagnosticsPlots(reglin, X_train, y_train)

print("\nResultados con datos de validacion\n")
getDiagnosticsPlots(reglin, X_test, y_test)

In [ ]:
# Instanciamos regresion con arbol de decisiones
tree = DecisionTreeRegressor(random_state=42)

# Ajustamos los datos a regresion linear
tree.fit(X_train, y_train)

# Creamos predicciones sobre set de entrenamiento y validacion
pred_train = tree.predict(X_train)
pred_test = tree.predict(X_test)

# Calculamos metricas - r2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f"R2 en datos de entrenamiento: {round(r2_train, 3)}")
print(f"R2 en datos de validacion: {round(r2_test, 3)}")

# Agregamos un espacio para claridad en resultados
print("\n")

# Calculamos metricas - mse
mse_train = mean_squared_error(y_train, pred_train)
mse_test =  mean_squared_error(y_test, pred_test)
print(f"MSE en datos de entrenamiento: {round(mse_train, 3)}")
print(f"MSE en datos de validacion: {round(mse_test, 3)}")

# Grafiquemos los resultados de la regresion
print("\nResultados con datos de entrenamiento\n")
getDiagnosticsPlots(tree, X_train, y_train)

print("\nResultados con datos de validacion\n")
getDiagnosticsPlots(tree, X_test, y_test)

In [ ]:
# Instanciamos regresion con arbol de decisiones
rf = RandomForestRegressor(random_state=42)

# Ajustamos los datos a regresion linear
rf.fit(X_train, y_train)

# Creamos predicciones sobre set de entrenamiento y validacion
pred_train = rf.predict(X_train)
pred_test = rf.predict(X_test)

# Calculamos metricas - r2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f"R2 en datos de entrenamiento: {round(r2_train, 3)}")
print(f"R2 en datos de validacion: {round(r2_test, 3)}")

# Agregamos un espacio para claridad en resultados
print("\n")

# Calculamos metricas - mse
mse_train = mean_squared_error(y_train, pred_train)
mse_test =  mean_squared_error(y_test, pred_test)
print(f"MSE en datos de entrenamiento: {round(mse_train, 3)}")
print(f"MSE en datos de validacion: {round(mse_test, 3)}")

# Grafiquemos los resultados de la regresion
print("\nResultados con datos de entrenamiento\n")
getDiagnosticsPlots(rf, X_train, y_train)

print("\nResultados con datos de validacion\n")
getDiagnosticsPlots(rf, X_test, y_test)

In [ ]:
# Instanciamos regresion con LightGBM
lgb = LGBMRegressor(random_state=42)

# Ajustamos los datos a regresion
lgb.fit(X_train, y_train)

# Creamos predicciones sobre set de entrenamiento y validacion
pred_train = lgb.predict(X_train)
pred_test = lgb.predict(X_test)

# Calculamos metricas - r2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f"R2 en datos de entrenamiento: {round(r2_train, 3)}")
print(f"R2 en datos de validacion: {round(r2_test, 3)}")

# Agregamos un espacio para claridad en resultados
print("\n")

# Calculamos metricas - mse
mse_train = mean_squared_error(y_train, pred_train)
mse_test =  mean_squared_error(y_test, pred_test)
print(f"MSE en datos de entrenamiento: {round(mse_train, 3)}")
print(f"MSE en datos de validacion: {round(mse_test, 3)}")

# Grafiquemos los resultados de la regresion
print("\nResultados con datos de entrenamiento\n")
getDiagnosticsPlots(lgb, X_train, y_train)

print("\nResultados con datos de validacion\n")
getDiagnosticsPlots(lgb, X_test, y_test)

#### Ajustemos algunos hiperparametros

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Especificamos cuadricula de busqueda de parametros
param_grid = {'max_depth': range(2,20,2),
              'n_estimators': range(10,110, 10),
              'min_samples_split': range(2,16,2),
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
             'criterion': ['friedman_mse'],
             'max_features': ['sqrt'],
             'bootstrap': [True, False]}

# Instanciamos el base y el de busqueda de parametros
rf = RandomForestRegressor(random_state=42)
rf_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, verbose=2, n_jobs = -1)

# Ajustar modelo de busqueda de parametros
rf_search.fit(X_train, y_train)

# Verificar mejores parametros
rf_search.best_params_

In [ ]:
# Creamos predicciones sobre set de entrenamiento y validacion
pred_train = rf_search.best_estimator_.predict(X_train)
pred_test = rf_search.best_estimator_.predict(X_test)

# Calculamos metricas - r2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f"R2 en datos de entrenamiento: {round(r2_train, 3)}")
print(f"R2 en datos de validacion: {round(r2_test, 3)}")

# Agregamos un espacio para claridad en resultados
print("\n")

# Calculamos metricas - mse
mse_train = mean_squared_error(y_train, pred_train)
mse_test =  mean_squared_error(y_test, pred_test)
print(f"MSE en datos de entrenamiento: {round(mse_train, 3)}")
print(f"MSE en datos de validacion: {round(mse_test, 3)}")

# Grafiquemos los resultados de la regresion
print("\nResultados con datos de entrenamiento\n")
getDiagnosticsPlots(rf_search.best_estimator_, X_train, y_train)

print("\nResultados con datos de validacion\n")
getDiagnosticsPlots(rf_search.best_estimator_, X_test, y_test)

In [ ]:
#### Datito del random forest!
fig, ax = plt.subplots(figsize=(12,8))
sns.barplot(X.columns, rf_search.best_estimator_.feature_importances_, ax=ax)

degrees = 90
plt.xticks(rotation=degrees)
plt.show()

#### BONO, Tensorflow y redes neuronales

In [ ]:
import tensorflow as tf

In [ ]:
# Capa de ingesta de datos
input_layer = tf.keras.layers.Input(shape=(X_train.shape[1],))

# Capa densa - todos los nodos conectados
dense_layer_1 = tf.keras.layers.Dense(16, activation='relu')(input_layer)

# Capa densa - todos los nodos conectados
dense_layer_2 = tf.keras.layers.Dense(16, activation='relu')(dense_layer_1)

# Capa de output
output = tf.keras.layers.Dense(1)(dense_layer_2)

# Compilar modelo
model = tf.keras.models.Model(inputs=input_layer, outputs=output)
model.compile(loss='mse', optimizer='adam')

In [ ]:
# Imprimir resumen de modelo
model.summary()

In [ ]:
# Entrenamos la red neuronal
history = model.fit(X_train, y_train, batch_size=8, epochs=50, verbose=1, validation_data=(X_test, y_test))

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))

ax.plot(history.history['val_loss'])
ax.plot(history.history['loss'])

In [ ]:
# Hacemos predicciones
pred_train = model.predict(X_train) 
pred_test = model.predict(X_test)

# Calculamos metricas - r2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)
print(f"R2 en datos de entrenamiento: {round(r2_train, 3)}")
print(f"R2 en datos de validacion: {round(r2_test, 3)}")

# Agregamos un espacio para claridad en resultados
print("\n")

# Calculamos metricas - mse
mse_train = mean_squared_error(y_train, pred_train)
mse_test =  mean_squared_error(y_test, pred_test)
print(f"MSE en datos de entrenamiento: {round(mse_train, 3)}")
print(f"MSE en datos de validacion: {round(mse_test, 3)}")